In [1]:
using CSV, MLJ, DataFrames, RDatasets

In [2]:
data = CSV.read("D:/Master/S7/R-Julia/Projet/CO2 Emissions_Canada.csv", DataFrames.DataFrame)
rename!(data, :"CO2 Emissions(g/km)"=> "CO2")
rename!(data, :"Vehicle Class"=>"Class")
rename!(data, :"Engine Size(L)"=>"Engine")
rename!(data, :"Fuel Type"=>"Fuel")
rename!(data, :"Fuel Consumption City (L/100 km)"=>"City")
rename!(data, :"Fuel Consumption Hwy (L/100 km)"=>"Hwy")
rename!(data, :"Fuel Consumption Comb (L/100 km)"=>"Comb")
rename!(data, :"Fuel Consumption Comb (mpg)"=>"mpg")

names(data)

12-element Vector{String}:
 "Make"
 "Model"
 "Class"
 "Engine"
 "Cylinders"
 "Transmission"
 "Fuel"
 "City"
 "Hwy"
 "Comb"
 "mpg"
 "CO2"

In [3]:
dropmissing(data)
nrow(data)
# pas de valeurs manquantes

7385

In [4]:
Qualitatives = ["Make","Model","Transmission","Fuel","Class"]
Quantitatives=["Engine","Cylinders","City","Hwy","Comb"]#,"mpg"]
Target = ["CO2"]
Features = append!(Qualitatives, Quantitatives)

10-element Vector{String}:
 "Make"
 "Model"
 "Transmission"
 "Fuel"
 "Class"
 "Engine"
 "Cylinders"
 "City"
 "Hwy"
 "Comb"

In [5]:
y = data.CO2
X = select(data, Not(:CO2))

Row,Make,Model,Class,Engine,Cylinders,Transmission,Fuel,City,Hwy,Comb,mpg
,String15,String,String31,Float64,Int64,String7,String1,Float64,Float64,Float64,Int64
1,ACURA,ILX,COMPACT,2.0,4,AS5,Z,9.9,6.7,8.5,33
2,ACURA,ILX,COMPACT,2.4,4,M6,Z,11.2,7.7,9.6,29
3,ACURA,ILX HYBRID,COMPACT,1.5,4,AV7,Z,6.0,5.8,5.9,48
4,ACURA,MDX 4WD,SUV - SMALL,3.5,6,AS6,Z,12.7,9.1,11.1,25
5,ACURA,RDX AWD,SUV - SMALL,3.5,6,AS6,Z,12.1,8.7,10.6,27
6,ACURA,RLX,MID-SIZE,3.5,6,AS6,Z,11.9,7.7,10.0,28
7,ACURA,TL,MID-SIZE,3.5,6,AS6,Z,11.8,8.1,10.1,28
8,ACURA,TL AWD,MID-SIZE,3.7,6,AS6,Z,12.8,9.0,11.1,25
9,ACURA,TL AWD,MID-SIZE,3.7,6,M6,Z,13.4,9.5,11.6,24


In [6]:
train, test = partition(eachindex(y), 0.7, shuffle=true, rng=424);
train[1:3]

3-element Vector{Int64}:
 2163
 1783
 5638

In [7]:
using NearestNeighbors

ArgumentError: ArgumentError: Package NearestNeighbors not found in current path.
- Run `import Pkg; Pkg.add("NearestNeighbors")` to install the NearestNeighbors package.

In [8]:
Xquali =  select(X,Qualitatives)
Xquantitatives = select(X,Quantitatives)

Row,Engine,Cylinders,City,Hwy,Comb
,Float64,Int64,Float64,Float64,Float64
1,2.0,4,9.9,6.7,8.5
2,2.4,4,11.2,7.7,9.6
3,1.5,4,6.0,5.8,5.9
4,3.5,6,12.7,9.1,11.1
5,3.5,6,12.1,8.7,10.6
6,3.5,6,11.9,7.7,10.0
7,3.5,6,11.8,8.1,10.1
8,3.7,6,12.8,9.0,11.1
9,3.7,6,13.4,9.5,11.6


In [9]:
stand1 = Standardizer();
Xquanti2=MLJ.transform(fit!(machine(stand1, Xquantitatives)), Xquantitatives)

┌ Info: Training machine(Standardizer(features = Symbol[], …), …).
└ @ MLJBase C:\Users\33750\.julia\packages\MLJBase\fEiP2\src\machines.jl:492


Row,Engine,Cylinders,City,Hwy,Comb
,Float64,Int64,Float64,Float64,Float64
1,-0.856663,4,-0.75895,-1.05271,-0.855684
2,-0.561279,4,-0.387551,-0.603161,-0.475391
3,-1.22589,4,-1.87315,-1.4573,-1.75456
4,0.251026,6,0.0409872,0.0262059,0.0431905
5,0.251026,6,-0.130428,-0.153613,-0.12967
6,0.251026,6,-0.187566,-0.603161,-0.337102
7,0.251026,6,-0.216136,-0.423342,-0.30253
8,0.398718,6,0.0695564,-0.0187489,0.0431905
9,0.398718,6,0.240972,0.206025,0.216051


In [10]:
yTrain = y[train]
XquantiTrain = Xquantitatives[train,:]
XquantiTest = Xquantitatives[test,:]

Row,Engine,Cylinders,City,Hwy,Comb
,Float64,Int64,Float64,Float64,Float64
1,2.0,4,9.4,6.5,8.1
2,3.0,6,15.2,9.6,12.7
3,3.0,6,13.2,9.5,11.5
4,5.7,8,17.3,11.7,14.8
5,2.4,4,9.6,6.5,8.2
6,3.5,6,14.0,9.9,12.4
7,3.5,6,9.6,7.9,8.8
8,3.0,6,14.9,9.8,12.6
9,5.0,8,14.8,9.9,12.6


In [11]:
using CategoricalArrays
Make2=convert(Vector, data[!,"Make"])
CategoricalArray(Make2)
Model2=convert(Vector, data[!,"Model"])
CategoricalArray(Model2)
Transmission2=convert(Vector, data[!,"Transmission"])
CategoricalArray(Transmission2)
Fuel_type2=convert(Vector, data[!,"Fuel"])
CategoricalArray(Fuel_type2)
Vehicle_class2=convert(Vector, data[!,"Class"])
CategoricalArray(Vehicle_class2)


using DataFrames
Xquali2=DataFrame(Make=categorical(Make2),Model=categorical(Model2),Transmission=categorical(Transmission2),Fuel_type=categorical(Fuel_type2),Vehicle_class=categorical(Vehicle_class2))

hot = OneHotEncoder()
mach = fit!(machine(hot, Xquali2))
W = MLJ.transform(mach, Xquali2)

┌ Info: Training machine(OneHotEncoder(features = Symbol[], …), …).
└ @ MLJBase C:\Users\33750\.julia\packages\MLJBase\fEiP2\src\machines.jl:492


┌ Info: Spawning 42 sub-features to one-hot encode feature :Make.
└ @ MLJModels C:\Users\33750\.julia\packages\MLJModels\L1gIK\src\builtins\Transformers.jl:878


┌ Info: Spawning 2053 sub-features to one-hot encode feature :Model.
└ @ MLJModels C:\Users\33750\.julia\packages\MLJModels\L1gIK\src\builtins\Transformers.jl:878


┌ Info: Spawning 27 sub-features to one-hot encode feature :Transmission.
└ @ MLJModels C:\Users\33750\.julia\packages\MLJModels\L1gIK\src\builtins\Transformers.jl:878


┌ Info: Spawning 5 sub-features to one-hot encode feature :Fuel_type.
└ @ MLJModels C:\Users\33750\.julia\packages\MLJModels\L1gIK\src\builtins\Transformers.jl:878


┌ Info: Spawning 16 sub-features to one-hot encode feature :Vehicle_class.
└ @ MLJModels C:\Users\33750\.julia\packages\MLJModels\L1gIK\src\builtins\Transformers.jl:878


Row,Make__ACURA,Make__ALFA ROMEO,Make__ASTON MARTIN,Make__AUDI,Make__BENTLEY,Make__BMW,Make__BUGATTI,Make__BUICK,Make__CADILLAC,Make__CHEVROLET,Make__CHRYSLER,Make__DODGE,Make__FIAT,Make__FORD,Make__GENESIS,Make__GMC,Make__HONDA,Make__HYUNDAI,Make__INFINITI,Make__JAGUAR,Make__JEEP,Make__KIA,Make__LAMBORGHINI,Make__LAND ROVER,Make__LEXUS,Make__LINCOLN,Make__MASERATI,Make__MAZDA,Make__MERCEDES-BENZ,Make__MINI,Make__MITSUBISHI,Make__NISSAN,Make__PORSCHE,Make__RAM,Make__ROLLS-ROYCE,Make__SCION,Make__SMART,Make__SRT,Make__SUBARU,Make__TOYOTA,Make__VOLKSWAGEN,Make__VOLVO,Model__124 SPIDER,Model__124 Spider,Model__1500,Model__1500 (MDS),Model__1500 4X4,Model__1500 4X4 (MDS),Model__1500 4X4 DIESEL,Model__1500 4X4 ECODIESEL,Model__1500 4X4 EcoDiesel,Model__1500 4X4 FFV,Model__1500 4X4 eTorque,Model__1500 Classic,Model__1500 Classic 4X4,Model__1500 Classic 4X4 EcoDiesel,Model__1500 Classic 4X4 FFV,Model__1500 Classic EcoDiesel,Model__1500 Classic FFV,Model__1500 DIESEL,Model__1500 ECODIESEL,Model__1500 ECODIESEL HFE,Model__1500 EcoDiesel,Model__1500 FFV,Model__1500 HFE,Model__1500 HFE eTorque,Model__1500 eTorque,Model__200,Model__200 AWD,Model__200 AWD FFV,Model__200 CONVERTIBLE,Model__200 CONVERTIBLE FFV,Model__200 FFV,Model__200 SEDAN,Model__200 SEDAN FFV,Model__228i,Model__228i CABRIOLET,Model__228i COUPE,Model__228i xDRIVE,Model__228i xDRIVE CABRIOLET,Model__228i xDRIVE COUPE,Model__230i CABRIOLET,Model__230i COUPE,Model__230i Cabriolet,Model__230i Coupe,Model__230i xDRIVE CABRIOLET,Model__230i xDRIVE COUPE,Model__230i xDrive Cabriolet,Model__230i xDrive Coupe,Model__300,Model__300 (MDS),Model__300 AWD,Model__300 AWD (MDS),Model__300 AWD FFV,Model__300 FFV,Model__300 SRT (MDS),Model__320i,Model__320i SEDAN,Model__320i xDRIVE,Model__320i xDRIVE SEDAN,⋯
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,⋯
1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,⋯
2,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,⋯
3,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,⋯
4,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0

In [12]:
X2=hcat(W,Xquanti2)

Row,Make__ACURA,Make__ALFA ROMEO,Make__ASTON MARTIN,Make__AUDI,Make__BENTLEY,Make__BMW,Make__BUGATTI,Make__BUICK,Make__CADILLAC,Make__CHEVROLET,Make__CHRYSLER,Make__DODGE,Make__FIAT,Make__FORD,Make__GENESIS,Make__GMC,Make__HONDA,Make__HYUNDAI,Make__INFINITI,Make__JAGUAR,Make__JEEP,Make__KIA,Make__LAMBORGHINI,Make__LAND ROVER,Make__LEXUS,Make__LINCOLN,Make__MASERATI,Make__MAZDA,Make__MERCEDES-BENZ,Make__MINI,Make__MITSUBISHI,Make__NISSAN,Make__PORSCHE,Make__RAM,Make__ROLLS-ROYCE,Make__SCION,Make__SMART,Make__SRT,Make__SUBARU,Make__TOYOTA,Make__VOLKSWAGEN,Make__VOLVO,Model__124 SPIDER,Model__124 Spider,Model__1500,Model__1500 (MDS),Model__1500 4X4,Model__1500 4X4 (MDS),Model__1500 4X4 DIESEL,Model__1500 4X4 ECODIESEL,Model__1500 4X4 EcoDiesel,Model__1500 4X4 FFV,Model__1500 4X4 eTorque,Model__1500 Classic,Model__1500 Classic 4X4,Model__1500 Classic 4X4 EcoDiesel,Model__1500 Classic 4X4 FFV,Model__1500 Classic EcoDiesel,Model__1500 Classic FFV,Model__1500 DIESEL,Model__1500 ECODIESEL,Model__1500 ECODIESEL HFE,Model__1500 EcoDiesel,Model__1500 FFV,Model__1500 HFE,Model__1500 HFE eTorque,Model__1500 eTorque,Model__200,Model__200 AWD,Model__200 AWD FFV,Model__200 CONVERTIBLE,Model__200 CONVERTIBLE FFV,Model__200 FFV,Model__200 SEDAN,Model__200 SEDAN FFV,Model__228i,Model__228i CABRIOLET,Model__228i COUPE,Model__228i xDRIVE,Model__228i xDRIVE CABRIOLET,Model__228i xDRIVE COUPE,Model__230i CABRIOLET,Model__230i COUPE,Model__230i Cabriolet,Model__230i Coupe,Model__230i xDRIVE CABRIOLET,Model__230i xDRIVE COUPE,Model__230i xDrive Cabriolet,Model__230i xDrive Coupe,Model__300,Model__300 (MDS),Model__300 AWD,Model__300 AWD (MDS),Model__300 AWD FFV,Model__300 FFV,Model__300 SRT (MDS),Model__320i,Model__320i SEDAN,Model__320i xDRIVE,Model__320i xDRIVE SEDAN,⋯
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,⋯
1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,⋯
2,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,⋯
3,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,⋯
4,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0

In [13]:
using CategoricalArrays
ycut=cut(y,10)

7385-element CategoricalArray{String,1,UInt32}:
 "Q2: [181.0, 199.0)"
 "Q4: [216.0, 230.0)"
 "Q1: [96.0, 181.0)"
 "Q6: [246.0, 260.0)"
 "Q5: [230.0, 246.0)"
 "Q5: [230.0, 246.0)"
 "Q5: [230.0, 246.0)"
 "Q6: [246.0, 260.0)"
 "Q7: [260.0, 278.0)"
 "Q3: [199.0, 216.0)"
 ⋮
 "Q3: [199.0, 216.0)"
 "Q4: [216.0, 230.0)"
 "Q4: [216.0, 230.0)"
 "Q3: [199.0, 216.0)"
 "Q4: [216.0, 230.0)"
 "Q5: [230.0, 246.0)"
 "Q5: [230.0, 246.0)"
 "Q5: [230.0, 246.0)"
 "Q6: [246.0, 260.0)"

In [14]:
KNNClassifier = @load KNNClassifier
v=[]
for i in [1,3,5]
    knnc = KNNClassifier(K=i, metric=Euclidean())
    classif = machine(knnc, X2, ycut)
    fit!(classif, rows=train)
    y2 = predict_mode(classif, rows=test)
    push!(v,[i,accuracy(y2, ycut[test])])
end

import NearestNeighborModels ✔

┌ Info: For silent loading, specify `verbosity=0`. 
└ @ Main C:\Users\33750\.julia\packages\MLJModels\L1gIK\src\loading.jl:159


UndefVarError: UndefVarError: `Euclidean` not defined

In [15]:
v2=[]
for i in [1,3,5]
    knnc = KNNClassifier(K=i, metric=Cityblock())
    classif = machine(knnc, X2, ycut)
    fit!(classif, rows=train)
    y2 = predict_mode(classif, rows=test)
    push!(v2,[i,accuracy(y2, ycut[test])])
end

UndefVarError: UndefVarError: `Cityblock` not defined

In [16]:
println("Euclidean",v)
println("Manhattan",v2)

Euclidean

Any[]
ManhattanAny[]


In [17]:
using Metrics, CategoricalArrays

ArgumentError: ArgumentError: Package Metrics not found in current path.
- Run `import Pkg; Pkg.add("Metrics")` to install the Metrics package.

In [18]:
ycut=cut(y,15)
Quantitatives=["Engine","Cylinders","City","Hwy","Comb"]
Xquantitatives = select(X,Quantitatives)
#enlever la taille du moteur améliore le modèle

Row,Engine,Cylinders,City,Hwy,Comb
,Float64,Int64,Float64,Float64,Float64
1,2.0,4,9.9,6.7,8.5
2,2.4,4,11.2,7.7,9.6
3,1.5,4,6.0,5.8,5.9
4,3.5,6,12.7,9.1,11.1
5,3.5,6,12.1,8.7,10.6
6,3.5,6,11.9,7.7,10.0
7,3.5,6,11.8,8.1,10.1
8,3.7,6,12.8,9.0,11.1
9,3.7,6,13.4,9.5,11.6


In [19]:
fuelType = data[!,"Fuel"]
function f(x)
    if x == "E" return 0
    end
    return 1
end
fuelType=f.(fuelType)
X2=hcat(fuelType,Xquanti2)

Row,x1,Engine,Cylinders,City,Hwy,Comb
,Int64,Float64,Int64,Float64,Float64,Float64
1,1,-0.856663,4,-0.75895,-1.05271,-0.855684
2,1,-0.561279,4,-0.387551,-0.603161,-0.475391
3,1,-1.22589,4,-1.87315,-1.4573,-1.75456
4,1,0.251026,6,0.0409872,0.0262059,0.0431905
5,1,0.251026,6,-0.130428,-0.153613,-0.12967
6,1,0.251026,6,-0.187566,-0.603161,-0.337102
7,1,0.251026,6,-0.216136,-0.423342,-0.30253
8,1,0.398718,6,0.0695564,-0.0187489,0.0431905
9,1,0.398718,6,0.240972,0.206025,0.216051


In [20]:
X2=hcat(fuelType,Xquanti2)

Row,x1,Engine,Cylinders,City,Hwy,Comb
,Int64,Float64,Int64,Float64,Float64,Float64
1,1,-0.856663,4,-0.75895,-1.05271,-0.855684
2,1,-0.561279,4,-0.387551,-0.603161,-0.475391
3,1,-1.22589,4,-1.87315,-1.4573,-1.75456
4,1,0.251026,6,0.0409872,0.0262059,0.0431905
5,1,0.251026,6,-0.130428,-0.153613,-0.12967
6,1,0.251026,6,-0.187566,-0.603161,-0.337102
7,1,0.251026,6,-0.216136,-0.423342,-0.30253
8,1,0.398718,6,0.0695564,-0.0187489,0.0431905
9,1,0.398718,6,0.240972,0.206025,0.216051


In [21]:
ycut=cut(y,12)

7385-element CategoricalArray{String,1,UInt32}:
 "Q3: [194.0, 208.0)"
 "Q5: [221.0, 232.0)"
 "Q1: [96.0, 177.0)"
 "Q7: [246.0, 258.0)"
 "Q6: [232.0, 246.0)"
 "Q5: [221.0, 232.0)"
 "Q6: [232.0, 246.0)"
 "Q7: [246.0, 258.0)"
 "Q8: [258.0, 272.0)"
 "Q4: [208.0, 221.0)"
 ⋮
 "Q4: [208.0, 221.0)"
 "Q4: [208.0, 221.0)"
 "Q4: [208.0, 221.0)"
 "Q4: [208.0, 221.0)"
 "Q4: [208.0, 221.0)"
 "Q6: [232.0, 246.0)"
 "Q6: [232.0, 246.0)"
 "Q6: [232.0, 246.0)"
 "Q7: [246.0, 258.0)"

In [34]:
using Metrics
using Distances
KNNClassifier = @load KNNClassifier
knn3 = KNNClassifier(K=3, metric=Cityblock())
classif = machine(knn3, X2, ycut)
fit!(classif, rows=train)
y2 = predict_mode(classif, rows=test)
#cm = confusion_matrix(y2, ycut[test])
println("Accuracy=", accuracy(y2, ycut[test]))# proportion de bien classifié
#println("Precision=", precision(y2, ycut[test]))
#println("Recall=", accuracy(y2, ycut[test]))#proportion de true positifs sur le nombre total de positif à trouver : TP+FN
println("erreur=", misclassification_rate(y2, ycut[test]))

┌ Info: For silent loading, specify `verbosity=0`. 
└ @ Main C:\Users\33750\.julia\packages\MLJModels\L1gIK\src\loading.jl:159


import NearestNeighborModels ✔
Accuracy=0.8744920993227991
erreur=0.1255079006772009


┌ Warning: The number and/or types of data arguments do not match what the specified model
│ supports. Suppress this type check by specifying `scitype_check_level=0`.
│ 
│ Run `@doc NearestNeighborModels.KNNClassifier` to learn more about your model's requirements.
│ 
│ Commonly, but non exclusively, supervised models are constructed using the syntax
│ `machine(model, X, y)` or `machine(model, X, y, w)` while most other models are
│ constructed with `machine(model, X)`.  Here `X` are features, `y` a target, and `w`
│ sample or class weights.
│ 
│ In general, data in `machine(model, data...)` is expected to satisfy
│ 
│     scitype(data) <: MLJ.fit_data_scitype(model)
│ 
│ In the present case:
│ 
│ scitype(data) = Tuple{Table{Union{AbstractVector{Continuous}, AbstractVector{Count}}}, AbstractVector{OrderedFactor{12}}}
│ 
│ fit_data_scitype(model) = Union{Tuple{Table{<:AbstractVector{<:Continuous}}, AbstractVector{<:Finite}}, Tuple{Table{<:AbstractVector{<:Continuous}}, AbstractVector{<:

In [23]:
using JLSO
JLSO.save("knn_CO2.jlso", :classifieur =>classif, :predireknn => predire)

ArgumentError: ArgumentError: Package JLSO not found in current path.
- Run `import Pkg; Pkg.add("JLSO")` to install the JLSO package.

In [31]:
function predire(Marque, Modele, Class, Taille_moteur, Cylindres, Transmission,Fuel ,City,Hwy,Comb)
    fuel =  f(Fuel)  
    donnees = DataFrame(Engine=Taille_moteur, Cylinders = Cylindres,City=City, Hwy=Hwy, Comb=Comb )
    donnees = MLJ.transform(fit!(machine(stand1, Xquantitatives)), donnees)
    x1 = DataFrame(fuel=fuel)
    donnees=hcat(x1, donnees)
    s = string.(predict_mode(classif, donnees))
    s = s[1]
    return "L'intevalle est " * s[4:length(s)-1] * "]"
    
end

predire (generic function with 1 method)

In [32]:
using JLD2
save_object("knn_classif.jld2", classif)

In [33]:
predire("ACURA","ILX","COMPACT",2,4,"AS5","Z",9.9,6.7,8.5)

┌ Info: Training machine(Standardizer(features = Symbol[], …), …).
└ @ MLJBase C:\Users\33750\.julia\packages\MLJBase\fEiP2\src\machines.jl:492


"L'intevalle est  [194.0, 208.0]"